In [123]:
import pandas as pd
import numpy as np
from fastapi import FastAPI
import json

Extraccion de datos.


In [145]:
#se crea una tabla IdPlataforma.
dfIdPlataforma=pd.DataFrame({"IdPlataforma":[1,2,3,4],
                            "Plataforma":["amazon","disney","hulu","netflix"]
                            })
dfIdPlataforma.head()

,IdPlataforma,Plataforma
0,1,amazon
1,2,disney
2,3,hulu
3,4,netflix


In [273]:
df1=pd.read_csv("Datasets/amazon_prime_titles.csv")
df1.drop(["director","country","date_added","rating","description"],axis=1,inplace=True)
df1.insert(0,"IdPlataforma",1)


df2=pd.read_csv("Datasets/disney_plus_titles.csv")
df2.drop(["director","country","date_added","rating","description"],axis=1,inplace=True)
df2.insert(0,"IdPlataforma",2)

df3=pd.read_csv("Datasets/hulu_titles.csv")
df3.drop(["director","country","date_added","rating","description"],axis=1,inplace=True)
df3.insert(0,"IdPlataforma",3)

with open('Datasets/netflix_titles.json') as file:
    data4 = json.load(file)
df4 = pd.DataFrame(data4)
df4.drop(["director","country","date_added","rating","description"],axis=1,inplace=True)
df4.insert(0,"IdPlataforma",4)


In [ ]:
df1.info()# columna cast tiene vacios
df2.info()# columna cast tiene vacios
df3.info()#columna cast todo esta vacio y columna duracion tiene vacios tb.
df4.info()#columna cast tiene vacios y tb la comna duracion tiene algunos vacios.

In [275]:
#reemplaza valores Nan por "" y creamos un nuevo IdVideo que hara unico la pelicula mas su año de realizacion mas su tipo
dftotal=pd.concat([df1,df2,df3,df4])
dftotal.reset_index(inplace=True,drop=True)
dftotal.replace(np.nan,"",inplace=True)
dftotal["IdVideo"]=dftotal["release_year"].apply(str)+dftotal["title"]+dftotal["type"]
dftotal.head(1)


,IdPlataforma,show_id,type,title,cast,release_year,duration,listed_in,IdVideo
0,1,s1,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",2014,113 min,"Comedy, Drama",2014The Grand SeductionMovie


In [ ]:
#for i in dfa[dfa.duplicated(False)].index:
#    for j in dfa[dfa.duplicated(False)].index:
#        if i!=j and dftotal.loc[i,"duration"]!=dftotal.loc[j,"duration"] and dftotal.loc[i,"IdVideo"]==dftotal.loc[j,"IdVideo"]:
#            print(dftotal.loc[i:i,("IdVideo","duration")])

In [277]:
dftotal[dftotal==""].count() # contamos las cantidas vacias, trataremos de reducirlo

IdPlataforma       0
show_id            0
type               0
title              0
cast            5321
release_year       0
duration         482
listed_in          0
IdVideo            0
dtype: int64

In [278]:
#Reemplaza los datos vacios en cast por algun duplicado con el mismo IdVideo
for i in dftotal[dftotal["cast"]==""].index: #dftotal[dftotal["cast"]==""].index nos da una lista con indices de vacio para la columna cast
    for j in dftotal[dftotal["IdVideo"].duplicated(False)].index:#dftotal[dftotal["IdVideo"].duplicated(False)].index nos da una lista con indices duplicados sin excluir los primeros encontrados.
        if i!=j and  dftotal.loc[j,"cast"]!="" and dftotal.loc[i,"IdVideo"]==dftotal.loc[j,"IdVideo"]:
            dftotal.loc[i,"cast"]=dftotal.loc[j,"cast"]
            break
            

In [279]:
#Reemplaza los datos vacios en duration por algun duplicado con el mismo IdVideo
for i in dftotal[dftotal["duration"]==""].index:#dftotal[dftotal["duration"]==""].index nos da una lista con indices de vacio para la columna duration
    for j in dftotal[dftotal["IdVideo"].duplicated(False)].index:#dftotal[dftotal["IdVideo"].duplicated(False)].index nos da una lista con indices duplicados sin excluir los primeros encontrados.
        if i!=j and  dftotal.loc[j,"duration"]!="" and dftotal.loc[i,"IdVideo"]==dftotal.loc[j,"IdVideo"]:
            dftotal.loc[i,"duration"]=dftotal.loc[j,"duration"]
            break

In [280]:
dftotal[dftotal==""].count()
# se logro reducir los vacios de la columna cast de 5321 a 5146
# se logro reducir los vacios de la columna duration de 482 a 451

IdPlataforma       0
show_id            0
type               0
title              0
cast            5146
release_year       0
duration         451
listed_in          0
IdVideo            0
dtype: int64

In [281]:
#se atomiza los datos de la columna cast para obtener la normalizacion F1
dfcast=dftotal.loc[:,("IdPlataforma","IdVideo","cast")]
ll=[]
mm=[]
nn=[]
for i in range(0,len(dfcast.iloc[:,2])):
    if "," in dfcast.iloc[i,2]:
        for j in list(dfcast.iloc[i,2].split(",")):
            ll.append(dfcast.iloc[i,0])
            mm.append(dfcast.iloc[i,1])
            nn.append(j)
    else:
        ll.append(dfcast.iloc[i,0])
        mm.append(dfcast.iloc[i,1])
        nn.append(dfcast.iloc[i,2])

nn=list(map(str.strip,nn))
dfcast=pd.DataFrame({"IdPlataforma":ll,
                "IdVideo":mm,
                "cast":nn})
dfcast.head(5)

,IdPlataforma,IdVideo,cast
0,1,2014The Grand SeductionMovie,Brendan Gleeson
1,1,2014The Grand SeductionMovie,Taylor Kitsch
2,1,2014The Grand SeductionMovie,Gordon Pinsent
3,1,2018Take Care Good NightMovie,Mahesh Manjrekar
4,1,2018Take Care Good NightMovie,Abhay Mahajan


In [282]:
#Se quitan duplicados en la tabla cast y se tiene listo la tabla cast
dfcast.drop_duplicates(inplace=True)
dfcast.reset_index(inplace=True,drop=True)
dfcast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120866 entries, 0 to 120865
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   IdPlataforma  120866 non-null  int64 
 1   IdVideo       120866 non-null  object
 2   cast          120866 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


In [284]:
#se atomiza los datos de la columna listed_in para obtener la normalizacion F1
dflisted=dftotal.loc[:,("IdPlataforma","IdVideo","listed_in")]
ll=[]
mm=[]
nn=[]
for i in range(0,len(dflisted.iloc[:,2])):
    if "," in dflisted.iloc[i,2]:
        for j in list(dflisted.iloc[i,2].split(",")):
            ll.append(dflisted.iloc[i,0])
            mm.append(dflisted.iloc[i,1])
            nn.append(j)
    else:
        ll.append(dflisted.iloc[i,0])
        mm.append(dflisted.iloc[i,1])
        nn.append(dflisted.iloc[i,2])

nn=list(map(str.strip,nn))
dflisted=pd.DataFrame({"IdPlataforma":ll,
                "IdVideo":mm,
                "listed_in":nn})
dflisted.head(5)

,IdPlataforma,IdVideo,listed_in
0,1,2014The Grand SeductionMovie,Comedy
1,1,2014The Grand SeductionMovie,Drama
2,1,2018Take Care Good NightMovie,Drama
3,1,2018Take Care Good NightMovie,International
4,1,2017Secrets of DeceptionMovie,Action


In [285]:
#se quitan los duplicados en la tabla listed y se tiene listo la tabla listed_in
dflisted.drop_duplicates(inplace=True)
dflisted.reset_index(inplace=True,drop=True)
dflisted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48303 entries, 0 to 48302
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IdPlataforma  48303 non-null  int64 
 1   IdVideo       48303 non-null  object
 2   listed_in     48303 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [287]:
#se tiene listo la tabla Video
dfVideo=dftotal.loc[:,("IdVideo","title","release_year","type")]
dfVideo.drop_duplicates(inplace=True)
dfVideo.reset_index(inplace=True,drop=True)
dfVideo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22608 entries, 0 to 22607
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IdVideo       22608 non-null  object
 1   title         22608 non-null  object
 2   release_year  22608 non-null  int64 
 3   type          22608 non-null  object
dtypes: int64(1), object(3)
memory usage: 706.6+ KB


In [288]:
#Se tienelisto la tabla plataforma
dfPlataforma=dftotal.loc[:,("IdPlataforma","IdVideo","duration")]

In [289]:
#Exportamos las tablas para ser cargads en SQL
dfIdPlataforma.to_csv("tables/IdPlataforma.csv")
dfPlataforma.to_csv("tables/Plataforma.csv")
dfcast.to_csv("tables/cast.csv",encoding="utf-8")
dflisted.to_csv("tables/listed.csv")
dfVideo.to_csv("tables/Video.csv")

In [311]:
dfPlataforma.info()
dfVideo.info()
dfcast.info()
dflisted.info()
dfIdPlataforma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IdPlataforma  22998 non-null  int64 
 1   IdVideo       22998 non-null  object
 2   duration      22998 non-null  object
dtypes: int64(1), object(2)
memory usage: 539.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22608 entries, 0 to 22607
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IdVideo       22608 non-null  object
 1   title         22608 non-null  object
 2   release_year  22608 non-null  int64 
 3   type          22608 non-null  object
dtypes: int64(1), object(3)
memory usage: 706.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120866 entries, 0 to 120865
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   IdPlataforma  120